In [1]:
!pip uninstall openpyxl
!pip install openpyxl 
!pip install pyreadr

Found existing installation: openpyxl 2.5.9
Uninstalling openpyxl-2.5.9:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/openpyxl-2.5.9.dist-info/*
    /usr/local/lib/python3.7/dist-packages/openpyxl/*
Proceed (y/n)? y
  Successfully uninstalled openpyxl-2.5.9
     |████████████████████████████████| 242 kB 11.4 MB/s 
     |████████████████████████████████| 361 kB 12.6 MB/s 


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pyreadr
import pandas as pd
import numpy as np
import os
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 1. Leer los datos de expresión de isoformas, genes

In [4]:
#Estos son los datos de expresión de isoforma (TCGA_tpm) de genes (TCGA_tpm_gn) las 
#información de tipo de tejido, norma vs tumoral (cond) y la información de la relación 
#de genes e isoformas (getBM).

# unlist Parsing_TCGA_LUAD:
result = pyreadr.read_r('gdrive/MyDrive/deepsf/data/input/LUAD/Parsing_luad.RData')
# result is a dictionary where keys are the name of objects.

print(result.keys()) # let's check what objects we got

#Tiene la lista de los RBPs conocidos (SF genes).
listRBPs = pd.read_excel('gdrive/MyDrive/deepsf/data/input/Table_S2_list_RBPs_eyras.xlsx', skiprows=2)

odict_keys(['TCGA_tpm', 'TCGA_tpm_gn', 'getBM', 'cond'])


In [5]:
# Shapes of the different raw datasets:
[print(y, x.shape) for y,x in result.items()]

TCGA_tpm (199169, 600)
TCGA_tpm_gn (60554, 600)
getBM (199169, 5)
cond (600, 1)


[None, None, None, None]

In [6]:
TCGA_tpm = result["TCGA_tpm"] #isoforma (transcrito) por tejido (valor de tpm)
TCGA_tpm.head()

TCGA_tpm_gn = result["TCGA_tpm_gn"] #genes por tejido (valor de tpm)
TCGA_tpm_gn.head()

getBM = result["getBM"] # matriz que te relaciona la isoforma (transcrito) con el gene_name
getBM.head()

cond = result["cond"] #para todos los pacientes (tejido) si son normal o tumoral.
cond.head()

listRBPs.head()

,TCGA-L9-A444-01A-21R-A24H-07_Tumor,TCGA-MP-A4T9-01A-11R-A24X-07_Tumor,TCGA-MP-A4TC-01A-11R-A24X-07_Tumor,TCGA-MP-A4TA-01A-21R-A24X-07_Tumor,TCGA-L4-A4E5-01A-11R-A24X-07_Tumor,TCGA-MP-A4T8-01A-11R-A24X-07_Tumor,TCGA-MP-A4T7-01A-11R-A24X-07_Tumor,TCGA-MP-A4SW-01A-21R-A24X-07_Tumor,TCGA-MP-A4SY-01A-21R-A24X-07_Tumor,TCGA-MP-A4SV-01A-11R-A24X-07_Tumor,TCGA-MN-A4N1-01A-11R-A24X-07_Tumor,TCGA-MN-A4N5-01A-11R-A24X-07_Tumor,TCGA-MN-A4N4-01A-12R-A24X-07_Tumor,TCGA-86-A4JF-01A-11R-A24X-07_Tumor,TCGA-86-A4P8-01A-11R-A24X-07_Tumor,TCGA-93-A4JN-01A-11R-A24X-07_Tumor,TCGA-93-A4JP-01A-11R-A24X-07_Tumor,TCGA-93-A4JQ-01A-11R-A24X-07_Tumor,TCGA-93-A4JO-01A-21R-A24X-07_Tumor,TCGA-97-A4LX-01A-11R-A24X-07_Tumor,TCGA-97-A4M0-01A-11R-A24X-07_Tumor,TCGA-97-A4M1-01A-11R-A24X-07_Tumor,TCGA-97-A4M3-01A-11R-A24X-07_Tumor,TCGA-97-A4M2-01A-12R-A24X-07_Tumor,TCGA-97-A4M5-01A-11R-A24X-07_Tumor,TCGA-97-A4M7-01A-11R-A24X-07_Tumor,TCGA-97-A4M6-01A-11R-A24X-07_Tumor,TCGA-86-A4P7-01A-11R-A24X-07_Tumor,TCGA-44-A4SU-01A-11R-A24X-07_Tumor,TCGA-53-A4EZ-01A-12R-A24X-07_Tumor,TCGA-44-A4SS-01A-11R-A24X-07_Tumor,TCGA-55-A48Z-01A-12R-A24X-07_Tumor,TCGA-55-A494-01A-11R-A24X-07_Tumor,TCGA-MP-A4TK-01A-11R-A24X-07_Tumor,TCGA-MP-A4TI-01A-21R-A24X-07_Tumor,TCGA-L9-A443-01A-12R-A24H-07_Tumor,TCGA-L4-A4E6-01A-11R-A24H-07_Tumor,TCGA-86-A4D0-01A-11R-A24H-07_Tumor,TCGA-62-A470-01A-11R-A24H-07_Tumor,TCGA-62-A472-01A-11R-A24H-07_Tumor,...,TCGA-44-5645-01A-01R-A278-07_Other,TCGA-44-3917-01A-01R-A278-07_Tumor,TCGA-44-6147-01B-06R-A277-07_Other,TCGA-44-2666-01B-02R-A277-07_Other,TCGA-44-4112-01B-06R-A277-07_Other,TCGA-44-3918-01B-02R-A277-07_Other,TCGA-44-3917-01B-02R-A277-07_Other,TCGA-44-2668-01B-02R-A277-07_Other,TCGA-44-6146-01B-04R-A277-07_Other,TCGA-44-2662-01B-02R-A277-07_Other,TCGA-44-5645-01B-04R-A277-07_Other,TCGA-44-2656-01B-06R-A277-07_Other,TCGA-49-AARQ-01A-11R-A41B-07_Tumor,TCGA-49-AARN-01A-21R-A41B-07_Tumor,TCGA-49-AARO-01A-12R-A41B-07_Tumor,TCGA-49-AAR4-01A-12R-A41B-07_Tumor,TCGA-55-A57B-01A-12R-A39D-07_Tumor,TCGA-99-AA5R-01A-11R-A39D-07_Tumor,TCGA-S2-AA1A-01A-12R-A39D-07_Tumor,TCGA-49-AARR-01A-11R-A41B-07_Tumor,TCGA-49-AAR3-01A-11R-A41B-07_Tumor,TCGA-73-A9RS-01A-11R-A41B-07_Tumor,TCGA-L9-A743-01A-43R-A39D-07_Tumor,TCGA-L9-A7SV-01A-11R-A39D-07_Tumor,TCGA-L9-A8F4-01A-11R-A39D-07_Tumor,TCGA-NJ-A7XG-01A-12R-A39D-07_Tumor,TCGA-49-AAR9-01A-21R-A41B-07_Tumor,TCGA-49-AARE-01A-11R-A41B-07_Tumor,TCGA-49-AAQV-01A-11R-A39D-07_Tumor,TCGA-49-AAR2-01A-11R-A39D-07_Tumor,TCGA-4B-A93V-01A-11R-A39D-07_Tumor,TCGA-49-AAR0-01A-21R-A39D-07_Tumor,TCGA-55-A48Y-01A-11R-A24H-07_Tumor,TCGA-L9-A5IP-01A-21R-A39D-07_Tumor,TCGA-L9-A50W-01A-12R-A39D-07_Tumor,TCGA-55-A490-01A-11R-A466-07_Tumor,TCGA-NJ-A4YG-01A-22R-A262-07_Tumor,TCGA-MP-A4TE-01A-22R-A466-07_Tumor,TCGA-55-7816-01A-11R-2170-07_Tumor,TCGA-44-3398-11B-01R-1758-07_Normal
rownames,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENST00000000233,92.642800,231.355000,197.86700,166.32700,239.04200,205.900000,105.2290,150.025000,152.680000,131.470000,311.018000,173.809000,225.0610,125.0270,113.01100,124.928000,120.67500,125.869000,135.863000,99.186200,121.221000,77.523500,112.272000,104.28000,76.820700,79.883700,99.499700,106.67400,124.8200,286.316000,185.573000,173.33500,235.23500,141.30000,169.49600,133.7460,119.90500,129.369000,107.952000,147.2090,...,12.19020,3.446790,1.179050,4.936000,3.817730,5.102240,4.90969,6.545040,3.041730,0.000000,0.456803,2.028930,182.30500,153.656000,220.273000,161.418000,89.23610,100.21700,61.387300,104.513000,138.191000,112.400000,117.10400,160.917000,114.394000,148.4390,212.16500,204.76500,192.334000,1.220090e+02,134.466000,183.4860,90.798100,87.662600,230.199000,87.5706,110.064000,155.17500,81.53430,100.87900
ENST00000000412,13.416100,37.246600,26.86280,32.23940,24.32130,27.363300,30.3292,30.646300,39.905200,34.483200,22.180300,26.773100,37.3299,45.6391,35.57240,26.814500,38.93880,31.608400,24.452400,46.183500,29.539900,25.333300,14.192600,36.23810,37.667100,34.210500,28.955300,25.6

,TCGA-L9-A444-01A-21R-A24H-07_Tumor,TCGA-MP-A4T9-01A-11R-A24X-07_Tumor,TCGA-MP-A4TC-01A-11R-A24X-07_Tumor,TCGA-MP-A4TA-01A-21R-A24X-07_Tumor,TCGA-L4-A4E5-01A-11R-A24X-07_Tumor,TCGA-MP-A4T8-01A-11R-A24X-07_Tumor,TCGA-MP-A4T7-01A-11R-A24X-07_Tumor,TCGA-MP-A4SW-01A-21R-A24X-07_Tumor,TCGA-MP-A4SY-01A-21R-A24X-07_Tumor,TCGA-MP-A4SV-01A-11R-A24X-07_Tumor,TCGA-MN-A4N1-01A-11R-A24X-07_Tumor,TCGA-MN-A4N5-01A-11R-A24X-07_Tumor,TCGA-MN-A4N4-01A-12R-A24X-07_Tumor,TCGA-86-A4JF-01A-11R-A24X-07_Tumor,TCGA-86-A4P8-01A-11R-A24X-07_Tumor,TCGA-93-A4JN-01A-11R-A24X-07_Tumor,TCGA-93-A4JP-01A-11R-A24X-07_Tumor,TCGA-93-A4JQ-01A-11R-A24X-07_Tumor,TCGA-93-A4JO-01A-21R-A24X-07_Tumor,TCGA-97-A4LX-01A-11R-A24X-07_Tumor,TCGA-97-A4M0-01A-11R-A24X-07_Tumor,TCGA-97-A4M1-01A-11R-A24X-07_Tumor,TCGA-97-A4M3-01A-11R-A24X-07_Tumor,TCGA-97-A4M2-01A-12R-A24X-07_Tumor,TCGA-97-A4M5-01A-11R-A24X-07_Tumor,TCGA-97-A4M7-01A-11R-A24X-07_Tumor,TCGA-97-A4M6-01A-11R-A24X-07_Tumor,TCGA-86-A4P7-01A-11R-A24X-07_Tumor,TCGA-44-A4SU-01A-11R-A24X-07_Tumor,TCGA-53-A4EZ-01A-12R-A24X-07_Tumor,TCGA-44-A4SS-01A-11R-A24X-07_Tumor,TCGA-55-A48Z-01A-12R-A24X-07_Tumor,TCGA-55-A494-01A-11R-A24X-07_Tumor,TCGA-MP-A4TK-01A-11R-A24X-07_Tumor,TCGA-MP-A4TI-01A-21R-A24X-07_Tumor,TCGA-L9-A443-01A-12R-A24H-07_Tumor,TCGA-L4-A4E6-01A-11R-A24H-07_Tumor,TCGA-86-A4D0-01A-11R-A24H-07_Tumor,TCGA-62-A470-01A-11R-A24H-07_Tumor,TCGA-62-A472-01A-11R-A24H-07_Tumor,...,TCGA-44-5645-01A-01R-A278-07_Other,TCGA-44-3917-01A-01R-A278-07_Tumor,TCGA-44-6147-01B-06R-A277-07_Other,TCGA-44-2666-01B-02R-A277-07_Other,TCGA-44-4112-01B-06R-A277-07_Other,TCGA-44-3918-01B-02R-A277-07_Other,TCGA-44-3917-01B-02R-A277-07_Other,TCGA-44-2668-01B-02R-A277-07_Other,TCGA-44-6146-01B-04R-A277-07_Other,TCGA-44-2662-01B-02R-A277-07_Other,TCGA-44-5645-01B-04R-A277-07_Other,TCGA-44-2656-01B-06R-A277-07_Other,TCGA-49-AARQ-01A-11R-A41B-07_Tumor,TCGA-49-AARN-01A-21R-A41B-07_Tumor,TCGA-49-AARO-01A-12R-A41B-07_Tumor,TCGA-49-AAR4-01A-12R-A41B-07_Tumor,TCGA-55-A57B-01A-12R-A39D-07_Tumor,TCGA-99-AA5R-01A-11R-A39D-07_Tumor,TCGA-S2-AA1A-01A-12R-A39D-07_Tumor,TCGA-49-AARR-01A-11R-A41B-07_Tumor,TCGA-49-AAR3-01A-11R-A41B-07_Tumor,TCGA-73-A9RS-01A-11R-A41B-07_Tumor,TCGA-L9-A743-01A-43R-A39D-07_Tumor,TCGA-L9-A7SV-01A-11R-A39D-07_Tumor,TCGA-L9-A8F4-01A-11R-A39D-07_Tumor,TCGA-NJ-A7XG-01A-12R-A39D-07_Tumor,TCGA-49-AAR9-01A-21R-A41B-07_Tumor,TCGA-49-AARE-01A-11R-A41B-07_Tumor,TCGA-49-AAQV-01A-11R-A39D-07_Tumor,TCGA-49-AAR2-01A-11R-A39D-07_Tumor,TCGA-4B-A93V-01A-11R-A39D-07_Tumor,TCGA-49-AAR0-01A-21R-A39D-07_Tumor,TCGA-55-A48Y-01A-11R-A24H-07_Tumor,TCGA-L9-A5IP-01A-21R-A39D-07_Tumor,TCGA-L9-A50W-01A-12R-A39D-07_Tumor,TCGA-55-A490-01A-11R-A466-07_Tumor,TCGA-NJ-A4YG-01A-22R-A262-07_Tumor,TCGA-MP-A4TE-01A-22R-A466-07_Tumor,TCGA-55-7816-01A-11R-2170-07_Tumor,TCGA-44-3398-11B-01R-1758-07_Normal
rownames,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TSPAN6,12.946603,99.586579,33.094212,24.888348,16.785836,58.617294,16.628493,39.093193,54.470680,29.935840,35.326858,70.387030,40.395540,14.276088,15.500407,43.459752,61.836000,15.836867,41.931758,25.746140,12.585566,31.317880,19.026949,16.843924,45.604548,38.125112,100.746428,14.555240,88.875279,42.187067,61.579530,76.005291,18.486841,24.241119,25.187488,26.370999,10.738681,32.503742,22.334178,23.522188,...,31.851092,4.366739,14.116490,8.650121,7.233805,12.197440,10.221412,6.872219,15.834611,15.328361,28.048303,22.746957,22.368517,33.644039,45.922780,23.401408,58.406929,17.197843,51.020808,49.972960,19.019155,14.183109,28.809037,16.052144,25.439482,64.463014,40.200019,48.259822,71.569371,22.461043,42.245320,14.352947,47.501790,12.086335,28.642888,8.790058,21.261057,46.544925,12.852150,21.819535
TNMD,0.021768,0.093324,0.000000,0.019322,0.000000,0.072461,0.000000,8.480220,0.046149,1.443796,0.091710,0.000000,0.000000,0.000000,0.045048,0.000000,0.087653,0.339289,0.000000,0.000000,0.000000,0.103638,0.000000,0.000000,0.070171,0.018228,0.071571,0.000000,0.017628,0.000000,0.000000,0.000000,0.00

,Transcript_ID,Gene_ID,Transcrip_name,Gene_name,Biotype
0,ENST00000000233,ENSG00000004059,ARF5-001,ARF5,protein_coding
1,ENST00000000412,ENSG00000003056,M6PR-001,M6PR,protein_coding
2,ENST00000000442,ENSG00000173153,ESRRA-002,ESRRA,protein_coding
3,ENST00000001008,ENSG00000004478,FKBP4-001,FKBP4,protein_coding
4,ENST00000001146,ENSG00000003137,CYP26B1-001,CYP26B1,protein_coding


,cond
0,Tumor
1,Tumor
2,Tumor
3,Tumor
4,Tumor


,HGNC symbol,TCGA ID,RNA compete matrix,type
0,A1CF,A1CF|29974,A1CF_00001,RBP/SF
1,ANKHD1,ANKHD1|54882,ANKHD1_00002,RBP/SF
2,CELF1,CELF1|10658,BRUNOL4_00004,RBP/SF
3,CELF2,CELF2|10659,BRUNOL4_00004,RBP/SF
4,CELF3,CELF3|11189,BRUNOL4_00004,RBP/SF


In [ ]:
cond.value_counts()
getBM[getBM['Gene_name'] == 'ARF5']
getBM['Biotype'].value_counts()[:5]
getBM.groupby(['Gene_name']).count()['Transcript_ID'].describe()

cond  
Tumor     516
Normal     59
Other      25
dtype: int64

,Transcript_ID,Gene_ID,Transcrip_name,Gene_name,Biotype
0,ENST00000000233,ENSG00000004059,ARF5-001,ARF5,protein_coding
42648,ENST00000415666,ENSG00000004059,ARF5-007,ARF5,protein_coding
70337,ENST00000459680,ENSG00000004059,ARF5-004,ARF5,retained_intron
73603,ENST00000463733,ENSG00000004059,ARF5-002,ARF5,retained_intron
76475,ENST00000467281,ENSG00000004059,ARF5-006,ARF5,processed_transcript
94555,ENST00000489673,ENSG00000004059,ARF5-003,ARF5,retained_intron


protein_coding             79930
processed_transcript       26977
retained_intron            26704
lincRNA                    13481
nonsense_mediated_decay    13409
Name: Biotype, dtype: int64

count    58684.000000
mean         3.393923
std          6.016443
min          1.000000
25%          1.000000
50%          1.000000
75%          4.000000
max        782.000000
Name: Transcript_ID, dtype: float64

# 2. Filtrar matriz de genes: sólo nos quedamos con los genes de splicing (SF = RBPs)

In [ ]:
# En primer lugar filtramos la matriz de genes y nos quedamos solo con los genes 
# de splicing (SF):
listRBPs_intersect = set(listRBPs['HGNC symbol']).intersection((set(TCGA_tpm_gn.index)))
print(len(listRBPs_intersect))

TCGA_tpm_gn_RBPs = TCGA_tpm_gn.loc[listRBPs_intersect] # matriz de genes SF por tejido (valor de tpm)
print(TCGA_tpm_gn_RBPs.shape)

1279
(1279, 600)


# 3. Eliminar isoformas únicas

In [ ]:
# Contamos el número de isoformas por gen:
num_iso_per_gn = getBM.groupby(['Gene_name']).count()['Transcript_ID']
print(num_iso_per_gn)

# Lista con los genes de la matriz de gn original  que solo tienen 1 isoforma:
listgn_uniqueiso = num_iso_per_gn[num_iso_per_gn == 1].index.to_list()
print("Number of genes with one isoform in the original gn df:", len(listgn_uniqueiso))

# Ahora tenemos que eliminar las isoformas únicas de la matriz de isoformas:
list_uniqueiso = getBM[getBM["Gene_name"].isin(listgn_uniqueiso)]["Transcript_ID"].to_list()
print("Number of unique isoform: ", len(list_uniqueiso))

# Finalmente, quitamos las isoformas únicas de la matriz de isoformas:
listISO_intersect = set(TCGA_tpm.index.to_list()) - set(list_uniqueiso) # isoformas con las que nos vamos a quedar
TCGA_tpm_without_uniqueiso = TCGA_tpm.loc[listISO_intersect]

print("\n", "Matriz de isoformas sin las isoformas únicas: ")
TCGA_tpm_without_uniqueiso.head()
TCGA_tpm_without_uniqueiso.shape

Gene_name
5S_rRNA       16
5_8S_rRNA      2
7SK            6
A1BG           5
A1BG-AS1       7
              ..
snoZ40         1
snoZ6          4
snosnR66       1
uc_338        36
yR211F11.2     1
Name: Transcript_ID, Length: 58684, dtype: int64
Number of genes with one isoform in the original gn df: 36740
Number of unique isoform:  36740

 Matriz de isoformas sin las isoformas únicas: 


,TCGA-L9-A444-01A-21R-A24H-07_Tumor,TCGA-MP-A4T9-01A-11R-A24X-07_Tumor,TCGA-MP-A4TC-01A-11R-A24X-07_Tumor,TCGA-MP-A4TA-01A-21R-A24X-07_Tumor,TCGA-L4-A4E5-01A-11R-A24X-07_Tumor,TCGA-MP-A4T8-01A-11R-A24X-07_Tumor,TCGA-MP-A4T7-01A-11R-A24X-07_Tumor,TCGA-MP-A4SW-01A-21R-A24X-07_Tumor,TCGA-MP-A4SY-01A-21R-A24X-07_Tumor,TCGA-MP-A4SV-01A-11R-A24X-07_Tumor,TCGA-MN-A4N1-01A-11R-A24X-07_Tumor,TCGA-MN-A4N5-01A-11R-A24X-07_Tumor,TCGA-MN-A4N4-01A-12R-A24X-07_Tumor,TCGA-86-A4JF-01A-11R-A24X-07_Tumor,TCGA-86-A4P8-01A-11R-A24X-07_Tumor,TCGA-93-A4JN-01A-11R-A24X-07_Tumor,TCGA-93-A4JP-01A-11R-A24X-07_Tumor,TCGA-93-A4JQ-01A-11R-A24X-07_Tumor,TCGA-93-A4JO-01A-21R-A24X-07_Tumor,TCGA-97-A4LX-01A-11R-A24X-07_Tumor,TCGA-97-A4M0-01A-11R-A24X-07_Tumor,TCGA-97-A4M1-01A-11R-A24X-07_Tumor,TCGA-97-A4M3-01A-11R-A24X-07_Tumor,TCGA-97-A4M2-01A-12R-A24X-07_Tumor,TCGA-97-A4M5-01A-11R-A24X-07_Tumor,TCGA-97-A4M7-01A-11R-A24X-07_Tumor,TCGA-97-A4M6-01A-11R-A24X-07_Tumor,TCGA-86-A4P7-01A-11R-A24X-07_Tumor,TCGA-44-A4SU-01A-11R-A24X-07_Tumor,TCGA-53-A4EZ-01A-12R-A24X-07_Tumor,TCGA-44-A4SS-01A-11R-A24X-07_Tumor,TCGA-55-A48Z-01A-12R-A24X-07_Tumor,TCGA-55-A494-01A-11R-A24X-07_Tumor,TCGA-MP-A4TK-01A-11R-A24X-07_Tumor,TCGA-MP-A4TI-01A-21R-A24X-07_Tumor,TCGA-L9-A443-01A-12R-A24H-07_Tumor,TCGA-L4-A4E6-01A-11R-A24H-07_Tumor,TCGA-86-A4D0-01A-11R-A24H-07_Tumor,TCGA-62-A470-01A-11R-A24H-07_Tumor,TCGA-62-A472-01A-11R-A24H-07_Tumor,...,TCGA-44-5645-01A-01R-A278-07_Other,TCGA-44-3917-01A-01R-A278-07_Tumor,TCGA-44-6147-01B-06R-A277-07_Other,TCGA-44-2666-01B-02R-A277-07_Other,TCGA-44-4112-01B-06R-A277-07_Other,TCGA-44-3918-01B-02R-A277-07_Other,TCGA-44-3917-01B-02R-A277-07_Other,TCGA-44-2668-01B-02R-A277-07_Other,TCGA-44-6146-01B-04R-A277-07_Other,TCGA-44-2662-01B-02R-A277-07_Other,TCGA-44-5645-01B-04R-A277-07_Other,TCGA-44-2656-01B-06R-A277-07_Other,TCGA-49-AARQ-01A-11R-A41B-07_Tumor,TCGA-49-AARN-01A-21R-A41B-07_Tumor,TCGA-49-AARO-01A-12R-A41B-07_Tumor,TCGA-49-AAR4-01A-12R-A41B-07_Tumor,TCGA-55-A57B-01A-12R-A39D-07_Tumor,TCGA-99-AA5R-01A-11R-A39D-07_Tumor,TCGA-S2-AA1A-01A-12R-A39D-07_Tumor,TCGA-49-AARR-01A-11R-A41B-07_Tumor,TCGA-49-AAR3-01A-11R-A41B-07_Tumor,TCGA-73-A9RS-01A-11R-A41B-07_Tumor,TCGA-L9-A743-01A-43R-A39D-07_Tumor,TCGA-L9-A7SV-01A-11R-A39D-07_Tumor,TCGA-L9-A8F4-01A-11R-A39D-07_Tumor,TCGA-NJ-A7XG-01A-12R-A39D-07_Tumor,TCGA-49-AAR9-01A-21R-A41B-07_Tumor,TCGA-49-AARE-01A-11R-A41B-07_Tumor,TCGA-49-AAQV-01A-11R-A39D-07_Tumor,TCGA-49-AAR2-01A-11R-A39D-07_Tumor,TCGA-4B-A93V-01A-11R-A39D-07_Tumor,TCGA-49-AAR0-01A-21R-A39D-07_Tumor,TCGA-55-A48Y-01A-11R-A24H-07_Tumor,TCGA-L9-A5IP-01A-21R-A39D-07_Tumor,TCGA-L9-A50W-01A-12R-A39D-07_Tumor,TCGA-55-A490-01A-11R-A466-07_Tumor,TCGA-NJ-A4YG-01A-22R-A262-07_Tumor,TCGA-MP-A4TE-01A-22R-A466-07_Tumor,TCGA-55-7816-01A-11R-2170-07_Tumor,TCGA-44-3398-11B-01R-1758-07_Normal
rownames,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ENST00000452544,0.014303,0.143303,0.129928,2.495270e-01,0.056998,0.215553,0.152035,0.049394,0.121187,0.24817,0.061399,0.085003,0.083136,0.25738,0.21669,0.066198,0.081761,0.157443,0.161198,0.044275,0.225709,1.872780e-02,0.101001,0.066243,0.064959,0.097660,0.049651,0.092410,0.054779,0.081417,0.118082,0.133704,0.243163,0.090759,0.126903,0.098581,0.043724,0.161491,0.035063,0.058624,...,0.257941,0.075227,4.031100,2.476560,4.412530,3.509770,3.677090,2.618630,6.485340,5.478610,5.858390,4.554260,0.063279,0.043408,0.072182,0.024565,0.184339,0.135448,0.130102,0.034375,0.055425,0.352483,0.179899,0.047991,0.163412,0.085606,0.235018,0.057385,0.059275,0.015992,0.351352,0.11455,0.031653,0.126599,0.057905,0.033098,0.04807,0.107264,0.070533,0.092513
ENST00000444816,1.639910,0.000000,0.905976,1.662730e-08,0.000000,2.333510,6.264120,0.000000,0.855907,0.00000,4.274960,0.000000,5.081430,1.95569,1.88634,0.507236,3.537120,4.108620,2.614460,0.805209,3.226130,1.005170e-08,0.000000,1.499490,0.345618,0.431183,4.926650,3.616570,2.345850,1.662360,1.576550,6.164280,3.562610,2.184160,0.000000,1.238540,0.997426,2.11763

(162429, 600)

# 4. Trasponemos las matriz para tener los datos de los pacientes en las filas

In [ ]:
TCGA_tpm_gn_RBPs = TCGA_tpm_gn_RBPs.T 
TCGA_tpm_without_uniqueiso = TCGA_tpm_without_uniqueiso.T

print("Número de SF genes (num features): ", TCGA_tpm_gn_RBPs.shape[1])
print("Número de isoformas únicas (num features): ", TCGA_tpm_without_uniqueiso.shape[1])

Número de SF genes (num features):  1279
Número de isoformas únicas (num features):  162429


In [ ]:
path = 'gdrive/MyDrive/deepsf/code_JS/folder_rawdata_processing'
#os.mkdir(path)

In [ ]:
# Matrices a guardar:

# Para 2-pipeline_unico_output_layer:

dictof_df2 = {'TCGA_tpm_gn_RBPs': TCGA_tpm_gn_RBPs, 
             'TCGA_tpm_without_uniqueiso': TCGA_tpm_without_uniqueiso, 
             'getBM': getBM}

file2 = open(path+'/2-pipeline_files.pkl', 'wb')
pickle.dump(dictof_df2, file2)
file2.close()


# Para 3-pipeline_API_output_layers.ipynb:

dictof_df3 = {'TCGA_tpm_gn': TCGA_tpm_gn.T,
             'TCGA_tpm_gn_RBPs': TCGA_tpm_gn_RBPs, 
             'TCGA_tpm_without_uniqueiso': TCGA_tpm_without_uniqueiso, 
             'getBM': getBM}

file3 = open(path+'/3-pipeline_files.pkl', 'wb')
pickle.dump(dictof_df3, file3)
file3.close()

# DE MOMENTO CON PICKLE NO ESTOY SIENDO CAPAZ DE LEER LOS DATOS EN 2 Y 3


#TCGA_tpm_gn.to_csv(path+'/TCGA_tpm_gn.csv') #Lista de genes original
#TCGA_tpm_gn_RBPs.to_csv(path+'/TCGA_tpm_gn_RBPs.csv')
#TCGA_tpm_without_uniqueiso.to_csv(path+'/TCGA_tpm_without_uniqueiso.csv')
#getBM.to_csv(path+'/getBM.csv')

# NOTA: USAR UN FORMATO PARA GUARDAR MAS EFICIENTE QUE CSV Y MODIFICAR ESO DE LOS DEMAS ARCHIVOS.
# ES MAS PUEDO HACER UN DICCIONARIO PARA CADA PIPELINE Y METER AHI LOS ARCHIVOS. LUEGO HACER UN README EN EL FOLDER.